In [25]:
pip install transformers sentence-transformers langchain faiss-cpu PyPDF2


In [10]:
pip install -U langchain-community

In [3]:
from huggingface_hub import login

# Replace 'your_huggingface_token' with your actual token
token = "AUTHTOKEN"

# Log in to Hugging Face with the token
login(token)


In [4]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Example usage
pdf_path = "fcw_final.pdf"  # Replace with your PDF file
pdf_text = extract_text_from_pdf(pdf_path)


In [7]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Split text into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_text(pdf_text)

# Load embeddings model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vector_store = FAISS.from_texts(documents, embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-7-200ab6460aea>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
def retrieve_relevant_context(query, vector_store, top_k=3):
    results = vector_store.similarity_search(query, k=top_k)
    return " ".join([res.page_content for res in results])

# Example query
query = "What is the functionality of Forward Collision Warning (FCW)?"
retrieved_context = retrieve_relevant_context(query, vector_store)


In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the LLaMA 2 model and tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Replace with your fine-tuned model if any
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add the [PAD] token if not already present
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

def generate_response(query, context):
    # Prepare the input text combining context and query
    input_text = f"Context: {context}\n\nQuery: {query}\n\nAnswer:"

    # Tokenize the input and move to GPU
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate the output with the model
    outputs = model.generate(inputs["input_ids"], max_length=512, temperature=0.7)

    # Decode the output tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Example query and context
query = "Give me full C++ code for forward collision warning with misra c++ 2023 complaince"
retrieved_context = "The context related to forward collision warning in automotive systems."

# Generate response
response = generate_response(query, retrieved_context)
print(response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00


In [4]:
pip install transformers sentence-transformers langchain faiss-cpu PyPDF2 streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.5 MB/s eta 0:00:00


In [28]:
!ngrok config add-authtoken AUTHTOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [27]:
!pip install streamlit pyngrok


In [29]:
from pyngrok import ngrok

ngrok.set_auth_token("AUTHTOKEN")

public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is running at {public_url}")


Streamlit app is running at NgrokTunnel: "https://0de5-35-199-177-74.ngrok-free.app" -> "http://localhost:8501"


In [32]:
!streamlit run apps.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.199.177.74:8501

  Stopping...
